In [1]:
from knowknow import *

In [2]:
from params import *

In [3]:
from load_db import db

loading variable anthro-wos-clean/_attributes from disk
loading variable anthro-wos-clean/groups from disk
loading variable anthro-wos-clean/group_reps from disk


In [4]:
focus_start = 1980
focus_end = 1990

In [5]:
total = 0
died = 0
died1 = 0

min_top_cited = float('inf')
all_avgbet = []
all_totals = []

for name in db.items('c'):
    tt = db.trend('c', name)
    
    all_totals.append( (name, tt.sum_between(focus_start,focus_end)) )
    
    # only count among works that are cited at least 5 times in focus_start-focus_end
    range1 = tt.sum_between(focus_start,focus_end) 
    if range1 < 5:
        continue        
        
    total += 1
    
    #range1 = min(min_top_cited, )
    B,D = tt.births_deaths(death_cutoff=0.1)
    B1,D1 = tt.births_deaths(death_cutoff=0)
    
    # did they die?
    if len(D):
        if D[-1] >= focus_start:
            died += 1
    # did they die?
    if len(D1):
        if D1[-1] >= focus_start:
            died1 += 1
            
    # how are they doing now?
    ab = tt.avg_between(2005,2015)
    all_avgbet.append(ab)

loading variable anthro-wos-clean/ind ___ c from disk
loading variable anthro-wos-clean/ind ___ c.fy from disk
loading variable anthro-wos-clean/ind ___ fy from disk


In [6]:
prop1 = died / total

In [7]:
prop2 = died1 / total

In [8]:
prop3 = sum( np.array(all_avgbet) < 1e-8 ) / len(all_avgbet)

In [9]:
prop1, prop2, prop3

(0.5224071702944942, 0.4714753165457391, 0.2112676056338028)

In [10]:
display(Markdown(f"""
{prop2:0.0%} of works that got at least 5 citations between {focus_start} and {focus_end}
    went at least a decade without being cited.
{prop3:0.0%} were not cited whatsoever between 2005 and 2015.
"""))


47% of works that got at least 5 citations between 1980 and 1990
    went at least a decade without being cited.
21% were not cited whatsoever between 2005 and 2015.


In [37]:
cy = db.by('c', 'fy').cits

In [38]:
cited = set()

for (c,fy), C in cy.items():
    if C <= 0:
        continue
    if focus_start <= fy <= focus_end:
        cited.add(c)

In [39]:
total_cited = len(cited)

In [40]:
total_cited

23410

In [ ]:
total = 0
died = 0
for name in cited:
    total += 1
    B,D = db.trend('c', name).births_deaths(death_cutoff=0.1)
    
    if len(D):
        died += 1

In [ ]:
prop2 = died / total

In [ ]:
prop1, prop2

In [ ]:
cited_after_1990 = set()

In [ ]:
for (c,fy), C in cy.items():
    if C <= 0:
        continue
    if 1990 < fy and c in cited:
        cited_after_1990.add(c)

In [ ]:
p1 = 1 - len(cited_after_1990) / len(cited)

In [ ]:
died = 0
total = 0

for c in cited:
    total += 1
    ct = db.trend('c', c)
    B,D =  ct.births_deaths( death_cutoff=0 )
    if len(D):
        died += 1

In [ ]:
p2 = died/total

In [ ]:
died = 0
total = 0

for c in cited:
    total += 1
    ct = db.trend('c', c)
    
    if ct.avg_between(2005,2015) >= 1:
        died += 1

In [ ]:
p3 = died/total

In [ ]:
p3

In [ ]:
all_avgbet = []

died = 0
total = 0

for c in db.items('c'):
    total += 1
    ct = db.trend('c', c)
    ab = ct.avg_between(2005,2015)
    all_avgbet.append(ab)

In [ ]:
p4 = sum( np.array(all_avgbet) > 1 ) / len(all_avgbet)

In [ ]:
p4

In [29]:
display(Markdown(f"""

The lifespan of a cited work is well described in aggregate by a lognormal distribution.
The work reaches its maximum after a few years, declines slowly for 5-10 years, eventually recieving its last citation.

A few statistics should be sufficient to illustrate that cited works are almost surely headed to the grave, never to be revived.
There were {total_cited:,} works cited for the first time in the years {focus_start} to {focus_end}.
{p1:0.0%} of these would not receive a single citation after 1990.
{p2:0.0%} of works cited in the 70s has gone at least a decade without being cited at all.
And only {p3:0.0%} received at least 1 citation per year on average between 2005 and 2015.
For comparison, {1-p4:0.0%} of works first cited between 2005 and 2015 received a yearly average of at least 1 citation in that decade.
"""))



The lifespan of a cited work is well described in aggregate by a lognormal distribution.
The work reaches its maximum after a few years, declines slowly for 5-10 years, eventually recieving its last citation.

A few statistics should be sufficient to illustrate that cited works are almost surely headed to the grave, never to be revived.
There were 23,410 works cited for the first time in the years 1980 to 1990.
6% of these would not receive a single citation after 1990.
66% of works cited in the 70s has gone at least a decade without being cited at all.
And only 7% received at least 1 citation per year on average between 2005 and 2015.
For comparison, 87% of works first cited between 2005 and 2015 received a yearly average of at least 1 citation in that decade.


# top 1%

In [30]:
len(all_totals)

56401

In [31]:
totns = np.array([x[1] for x in all_totals])

top1q = np.quantile(totns, 0.99)
top1 = [x for x,c in all_totals if c >= top1q]

In [55]:
all_avgbet = []

died = 0
total = 0

for c in top1:
    total += 1
    ct = db.trend('c', c)
    ab = ct.avg_between(2015,2020)
    #sb = ct.sum_between(2015,2020)
    all_avgbet.append(ab)

In [56]:
len(top1)

619

In [57]:
len(all_avgbet)

619

In [58]:
p1 = sum( np.array(all_avgbet) >= 1 ) / len(all_avgbet)

In [59]:
display(Markdown(f"""
{len(all_totals):,} works in this dataset were cited at some point between {focus_start} and {focus_end}.
The top 1% consists of {len(all_avgbet)} cited works which have at least {top1q:0.0f} citations in this decade.
Of these {p1:0.1%} got at least 1 citation per year 2015 - 2020.
"""))


56,401 works in this dataset were cited at some point between 1980 and 1990.
The top 1% consists of 619 cited works which have at least 18 citations in this decade.
Of these 58.5% got at least 1 citation per year 2015 - 2020.
